# Base Folder

In [107]:
import os

BASE_FOLDER = '2024-07-14 22-10-11'

base_folder = os.path.join('output', BASE_FOLDER)

### Read Files

In [108]:
import json

class Word:
    def __init__(self, word_json, filename):
        name, _ = os.path.splitext(os.path.basename(filename))

        self.file_index = int(name)
        self.probability = word_json['probability']
        self.word = word_json['word']

def parse_json(filename):
    retVal = []

    with open(filename, 'r') as f:
        log = json.load(f)

    for word in log['words']:
        retVal.append(Word(word, filename))
    
    return retVal

In [109]:
folder_dict = {}

for span_bool in os.listdir(base_folder):
    sb_split = span_bool.split()
    key = (int(sb_split[0]), bool(sb_split[0]))
    #print(str(key))

    words = []

    foldername = os.path.join(base_folder, span_bool)
    for filename in os.listdir(foldername):
        if filename.endswith('.json'):
            words.extend(parse_json(os.path.join(foldername, filename)))
    
    folder_dict[key] = sorted(words, key=lambda o: o.file_index)

### Color Printing

In [110]:
from IPython.display import display, HTML

# fg is in the format: '#RRGGBB'.  alpha doesn't seem to be supported
def print_colored(text, fg):
    output = "<span style=\"color:%s\">%s</span>" % (fg, text)
    display(HTML(output))

# items=[(text, fg), (text, fg) ...]
def print_colored_tuples(items):
    output = "".join("<span style=\"color:%s\">%s</span><br>" % (fg, text) for text, fg in items)
    display(HTML(output))

# Converts a color from hex format to rgb
def hex_to_rgb(hex):
    return tuple(int(hex[i:i+2], 16) for i in (0, 2, 4))

# Converts a color from rgb format back to hex
def rgb_to_hex(rgb):   
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

# Linearly interpolates between two colors
def lerp(c1, c2, pct):
    r1, g1, b1 = hex_to_rgb(c1[1:])   # Remove '#' and convert to RGB
    r2, g2, b2 = hex_to_rgb(c2[1:])
    
    r = int((r2 - r1) * pct + r1)  # Interpolate for each channel.
    g = int((g2 - g1) * pct + g1)
    b = int((b2 - b1) * pct + b1)
    
    return rgb_to_hex((r, g, b))  # Convert back to HEX and return.

print_colored('there', '#FF0000')
print_colored('calculated color', lerp('#FF0000', '#0000FF', 0.35))

In [111]:
# Takes two lists of tuples, iterates both lists in a single pass
# Replaces entries with None when the list lengths are uneven
def zip_longest_custom(col1, col2):
    len1 = len(col1)
    len2 = len(col2)
    maxlen = max(len1, len2)
    
    for i in range(maxlen):  # Loop through the length of the longest list.
        val1 = col1[i] if i < len1 else None
        val2 = col2[i] if i < len2 else None
        
        yield val1, val2

# Each column is a list of tuples:
# col=[(text, fg), (text, fg) ...]
def print_colored_columns(col1, col2):
    # Use zip_longest instead of chain for proper handling of uneven list lengths
    combined = zip_longest_custom(col1, col2)

    output = ""
    for item1, item2 in combined:
        if item1 is not None and item2 is not None:
            text1, fg1 = item1
            text2, fg2 = item2
            
            # Use a paragraph tag to separate each line of text
            output += "<p style=\"display:inline;\">" + \
                      "<span style=\"color:%s\">%s</span> | " % (fg1, text1) + \
                      "<span style=\"color:%s\">%s</span>" % (fg2, text2) + \
                      "</p>"

        elif item1 is not None:  # Handle the case where only col1 has an element
            text1, fg1 = item1
            output += "<p style=\"display:inline;\">" + \
                      "<span style=\"color:%s\">%s</span> | " % (fg1, text1) + \
                      "<span style=\"color:transparent;\" title=\"None\"> </span>" + \
                      "</p>"

        elif item2 is not None:  # Handle the case where only col2 has an element
            text2, fg2 = item2
            output += "<p style=\"display:inline;\"><span style=\"color:transparent;\" title=\"None\"> </span> | " + \
                      "<span style=\"color:%s\">%s</span>" % (fg2, text2) + \
                      "</p>"
    
    display(HTML(output))

col1 = [("Hello", "red"), ("World", "green")]
col2 = [("Python", "blue"), ("Development", "orange"), ("Fun!", "purple")]
print_colored_columns(col1, col2)

The above function tries to draw in two columns, but just makes a single line

Replace the html with this multi column version

In [112]:
from ipywidgets import VBox

html_column1 = """<div style="width: 100%;">Column 1 Content Here</div>"""
html_column2 = """<div style="width: 100%;">Column 2 Content Here</div>"""
html_column3 = """<div style="width: 100%;">Column 3 Content Here</div>"""

combined_html = f"""
<div style="display:flex;">
    {html_column1}
    {html_column2}
    {html_column3}
</div>
"""

display(HTML(combined_html))